In [1]:
push!(LOAD_PATH, "../src/")

using DataFrames
using Molecules
using Unitful
using Netsuriki

In [2]:
CH2_geom = """
C                  0.00000000    0.00000000    0.10308800
H                  0.00000000    0.99645900   -0.30926300
H                  0.00000000   -0.99645900   -0.30926300
"""
CH2_mol = Molecule(CH2_geom, 0, 3)
CH2_ν̃s = [1058.1217, 3125.2710, 3362.8179] .* 1.0u"1/cm";

In [3]:
# Electronic
q_el   = Electronic.q(CH2_mol)
Am_el  = Electronic.Am(q_el)
Gm_el  = Electronic.Gm(q_el)
Um_el  = Electronic.Um()
Hm_el  = Electronic.Hm()
Sm_el  = Electronic.Sm(q_el)
CVm_el = Electronic.CVm()
Cpm_el = Electronic.Cpm();

In [4]:
# Translational
q_tr   = Translational.q(CH2_mol)
Am_tr  = Translational.Am(q_tr)
Gm_tr  = Translational.Gm(q_tr)
Um_tr  = Translational.Um()
Hm_tr  = Translational.Hm()
Sm_tr  = Translational.Sm(q_tr)
CVm_tr = Translational.CVm()
Cpm_tr = Translational.Cpm();

In [5]:
# Rotational
q_rot   = Rotational.q(CH2_mol)
Am_rot  = Rotational.Am(q_rot)
Gm_rot  = Rotational.Gm(q_rot)
Um_rot  = Rotational.Um()
Hm_rot  = Rotational.Hm()
Sm_rot  = Rotational.Sm(q_rot)
CVm_rot = Rotational.CVm()
Cpm_rot = Rotational.Cpm();

In [6]:
# Vibrational
q_vib   = Vibrational.q(CH2_ν̃s)
Am_vib  = Vibrational.Am(q_vib)
Gm_vib  = Vibrational.Gm(q_vib)
Um_vib  = Vibrational.Um(CH2_ν̃s)
Hm_vib  = Vibrational.Hm(CH2_ν̃s)
Sm_vib  = Vibrational.Sm(CH2_ν̃s)
CVm_vib = Vibrational.CVm(CH2_ν̃s)
Cpm_vib = Vibrational.Cpm(CH2_ν̃s);

In [7]:
# Total
q_tot   = q_el   * q_tr   * q_rot   * q_vib
Am_tot  = Am_el  + Am_tr  + Am_rot  + Am_vib
Gm_tot  = Gm_el  + Gm_tr  + Gm_rot  + Gm_vib
Um_tot  = Um_el  + Um_tr  + Um_rot  + Um_vib
Hm_tot  = Hm_el  + Hm_tr  + Hm_rot  + Hm_vib
Sm_tot  = Sm_el  + Sm_tr  + Sm_rot  + Sm_vib
CVm_tot = CVm_el + CVm_tr + CVm_rot + CVm_vib
Cpm_tot = Cpm_el + Cpm_tr + Cpm_rot + Cpm_vib;

In [8]:
function thermo_J()
    DataFrame(
        Item = ["Total", "Electronic", "Translational", "Rotational", "Vibrational"],
        Am   = [Am_tot,  Am_el,  Am_tr,  Am_rot,  Am_vib],
        Gm   = [Gm_tot,  Gm_el,  Gm_tr,  Gm_rot,  Gm_vib],
        Um   = [Um_tot,  Um_el,  Um_tr,  Um_rot,  Um_vib],
        Hm   = [Hm_tot,  Hm_el,  Hm_tr,  Hm_rot,  Hm_vib],
        Sm   = [Sm_tot,  Sm_el,  Sm_tr,  Sm_rot,  Sm_vib],
        CVm  = [CVm_tot, CVm_el, CVm_tr, CVm_rot, CVm_vib],
        Cpm  = [Cpm_tot, Cpm_el, Cpm_tr, Cpm_rot, Cpm_vib]
    )
end

@show thermo_J();

thermo_J() = 5×8 DataFrame
 Row │ Item           Am                Gm                Um               Hm               Sm                   CVm                  Cpm
     │ String         Quantity…         Quantity…         Quantity…        Quantity…        Quantity…            Quantity…            Quantity…
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Total          -50663.8 J mol⁻¹  -48184.8 J mol⁻¹  52650.4 J mol⁻¹  55129.4 J mol⁻¹  195.129 J K⁻¹ mol⁻¹  26.2737 J K⁻¹ mol⁻¹  34.5882 J K⁻¹ mol⁻¹
   2 │ Electronic     -2723.41 J mol⁻¹  -2723.41 J mol⁻¹      0.0 J mol⁻¹      0.0 J mol⁻¹  9.13437 J K⁻¹ mol⁻¹      0.0 J K⁻¹ mol⁻¹      0.0 J K⁻¹ mol⁻¹
   3 │ Translational  -38524.5 J mol⁻¹  -36045.5 J mol⁻¹  3718.44 J mol⁻¹  6197.39 J mol⁻¹  141.683 J K⁻¹ mol⁻¹  12.4717 J K⁻¹ mol⁻¹  20.7862 J K⁻¹ mol⁻¹
   4 │ Rotational     -9400.82 J mol⁻¹  -9400.82 J mol⁻¹  3718.44 J mol⁻¹ 

In [9]:
function thermo_cal()
    df = thermo_J()
    map([:Am, :Gm, :Um, :Hm]) do x
        select!(df, :, x => (x -> x .|> u"kcal/mol") => x)
    end
    map([:Sm, :CVm, :Cpm]) do x
        select!(df, :, x => (x -> x .|> u"cal/mol/K") => x)
    end
    df
end

@show thermo_cal();

thermo_cal() = 5×8 DataFrame
 Row │ Item           Am                     Gm                     Um                   Hm                   Sm                       CVm                    Cpm
     │ String         Quantity…              Quantity…              Quantity…            Quantity…            Quantity…                Quantity…              Quantity…
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Total            -12.1089 kcal mol⁻¹    -11.5164 kcal mol⁻¹   12.5838 kcal mol⁻¹   13.1762 kcal mol⁻¹     46.637 cal K⁻¹ mol⁻¹  6.27957 cal K⁻¹ mol⁻¹  8.26678 cal K⁻¹ mol⁻¹
   2 │ Electronic      -0.650911 kcal mol⁻¹   -0.650911 kcal mol⁻¹       0.0 kcal mol⁻¹       0.0 kcal mol⁻¹    2.18317 cal K⁻¹ mol⁻¹      0.0 cal K⁻¹ mol⁻¹      0.0 cal K⁻¹ mol⁻¹
   3 │ Translational    -9.20757 kcal mol⁻¹    -8.61509 kcal mol⁻¹  0.888727 kcal mol⁻¹   1.48121 kc